In [ ]:
import intake
import matplotlib.pyplot as plt
cat = intake.open_catalog("https://data.nextgems-h2020.eu/online.yaml")
cat.ICON.ngc3028.to_dask().tas.mean(dim="cell").plot()
plt.savefig("images/tas_ngc3028_minimal.png")

In [ ]:
import intake
import healpy as hp
cat = intake.open_catalog("https://data.nextgems-h2020.eu/online.yaml")
hp.mollview(cat.ICON.ngc3028.to_dask().tas.isel(time=0), flip='geo', nest=True)

plt.savefig("images/healpix_l0.png")

In [ ]:
import intake
import healpy as hp
import matplotlib.pyplot as plt
cat = intake.open_catalog("https://data.nextgems-h2020.eu/online.yaml")
hp.mollview(cat.ICON.ngc3028(zoom=5).to_dask().tas.isel(time=0), flip='geo', nest=True, cmap='inferno')

plt.savefig("images/healpix_l5.png")

In [ ]:
import intake
import healpy as hp
import pandas as pd

cat = intake.open_catalog("https://data.nextgems-h2020.eu/online.yaml")
zoom = 5
phi=([53.5, 60.3251229])
theta = ([10, 24.7303234])
cells = hp.ang2pix(theta=theta, phi=phi, nside=hp.order2nside(zoom), lonlat=True, nest=True)
tas = cat.ICON.ngc3028(zoom=zoom).to_dask().tas.isel(cell = cells)
df = pd.DataFrame( dict(Hamburg=tas.isel(cell=0), Korpilampi=tas.isel(cell=1)), index=tas.time)
df.plot()

In [ ]:
from easygems.healpix import attach_coords
ds = cat.ICON.ngc3028(zoom=5).to_dask().pipe(attach_coords)
latgroups = ds.tas.isel(time=0).groupby('lat')
latgroups.mean().plot()
plt.xlim((-90,90))
plt.savefig("images/tas_vs_lat.png")

In [ ]:
from easygems.healpix import attach_coords
ds = cat.ICON.ngc3028(zoom=5).to_dask().pipe(attach_coords)
longroups = ds.tas.isel(time=0).groupby('lon')
longroups.mean().plot()


In [ ]:
import seaborn as sns
sns.violinplot(data=df)

In [ ]:
world=cat.ICON.ngc3028(zoom=zoom).to_dask().tas.isel(time=0)
world[cells]= 4e2
hp.mollview(world, flip='geo', nest=True, cmap='inferno')
plt.savefig("images/position_check.png")

In [ ]:
plt.figure(figsize=(7,7))
sns.scatterplot(df, x="Hamburg",y="Korpilampi", hue=df.index.month, palette=sns.color_palette("husl", 12))
plt.xlim(plt.ylim())
plt.savefig("images/scatterplot.png")

In [ ]:
df.index.month

In [ ]:
from easygems.healpix import attach_coords
ds = cat.ICON.ngc3028(zoom=5).to_dask().pipe(attach_coords)
latgroups = ds.ta.isel(time=0).groupby('lat')
lgm = latgroups.mean()
lgm.plot(cmap="inferno")
plt.xlim((-90,90))
plt.savefig("images/vert_sec_bad.png")

In [ ]:
lgm.plot(cmap="inferno")
plt.ylim(plt.ylim()[::-1])
plt.savefig("images/vert_sec_flipped.png")

In [ ]:
plt.pcolormesh(lgm, cmap="inferno")

In [ ]:
hera_cat = intake.open_catalog("https://tcodata.mpimet.mpg.de/internal.yaml")
hera = hera_cat.HERA5.to_dask().pipe(attach_coords)
# Contains modified Copernicus Climate Change Service information 2023. Neither the European Commission nor ECMWF is responsible for any use that may be made of the Copernicus information or data it contains.

In [ ]:
import numpy as np
t_mean_model = ds.sel(time=slice("2020-02-01", "2020-02-29")).tas.mean(dim="time")
t_mean_obs = hp.ud_grade(np.squeeze(hera.sel(time="2020-02")["2t"].values), hp.npix2nside(len(t_mean_model)), order_in = "nest", order_out = "nest")

hp.mollview(t_mean_model-t_mean_obs, flip="geo", nest=True, cmap="RdBu_r", min=-15, max=15)

In [ ]:
timeslice = slice("2020-02-01", "2023-01-31")
t_mean_model = ds.sel(time=timeslice).tas.mean(dim="time")
t_mean_obs = hp.ud_grade(np.squeeze(hera.sel(time=timeslice)['2t'].mean(dim="time").values), hp.npix2nside(len(t_mean_model)), order_in = "nest", order_out = "nest")

hp.mollview(t_mean_model-t_mean_obs, flip="geo", nest=True, cmap="RdBu_r", min=-5, max=5)

In [ ]:
params = dict(flip='geo', nest=True, cmap='inferno', min=216, max=304)
hp.mollview(t_mean_model, **params)
plt.title("model")
hp.mollview(t_mean_obs, **params)
plt.title("observation")


In [ ]:
print (hera.attrs['acknowledgment'])

In [ ]:
hera.time